# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def clear_data(string1):
    if re.search(r'\[[a-z]* [0-9]+\]', string1) is None:
        return string1
    else:
        return string1.replace(re.search(r'\[[a-z]* [0-9]+\]', string1).group(), '')

def get_area(team):
    for each in list(nhl_cities.index.values):
        if team in each:
            return nhl_cities.at[each, 'Metropolitan area']

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['NHL'] = cities['NHL'].apply(lambda x: clear_data(x))
nhl_cities = cities[['Metropolitan area', 'NHL']].set_index('NHL')
nhl_cities = nhl_cities.drop(['—', ''], axis=0)
nhl_df = nhl_df[nhl_df['year'] == 2018].drop([0, 9, 18, 26], axis=0) # get only 2018 stats
population = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
population = population.set_index('Metropolitan area')
nhl_df['team'] = nhl_df['team'].apply(lambda x: x[:-1].strip() if x.endswith("*") else x.strip())
nhl_df['area'] = nhl_df['team'].apply(lambda x: x.split(" ")[-1])
nhl_df['area'] = nhl_df['area'].apply(lambda x: get_area(x))
out = []
for group, frame in nhl_df.groupby('area'):
    total_wins = np.sum(pd.to_numeric(frame['W']))
    total_losses = np.sum(pd.to_numeric(frame['L']))
    total_matches = total_wins + total_losses
    ratio = total_wins / total_matches
    out_dict = {
        'Area': group,
        'Ratio': ratio
    }
    out.append(out_dict)
new_df = pd.DataFrame(out)
new_df = new_df.set_index('Area')
out_df = pd.merge(new_df, population, how="inner", left_index=True, right_index=True)
out_df['Population (2016 est.)[8]'] = pd.to_numeric(out_df['Population (2016 est.)[8]'])

def nhl_correlation(): 
    population_by_region = out_df['Population (2016 est.)[8]'].to_list()
    win_loss_by_region = out_df['Ratio'].to_list()

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return np.float64(stats.pearsonr(population_by_region, win_loss_by_region)[0])

def get_nhl_data():
    return out_df

# import pandas as pd
# import numpy as np
# import scipy.stats as stats
# import re

# def clear_data(string1):
#     if re.search(r'\[[a-z]* [0-9]+\]', string1) is None:
#         return string1
#     else:
#         return string1.replace(re.search(r'\[[a-z]* [0-9]+\]', string1).group(), '')


# def get_area(team):
#     for each in list(nhl_cities.index.values):
#         if team in each:
#             return nhl_cities.at[each, 'Metropolitan area']


# nhl_df=pd.read_csv("assets/nhl.csv")
# cities=pd.read_html("assets/wikipedia_data.html")[1]
# cities=cities.iloc[:-1,[0,3,5,6,7,8]]
# cities['NHL'] = cities['NHL'].apply(lambda x: clear_data(x))
# nhl_cities = cities[['Metropolitan area', 'NHL']].set_index('NHL')
# nhl_cities = nhl_cities.drop(['—', ''], axis=0)
# nhl_df = nhl_df[nhl_df['year'] == 2018].drop([0, 9, 18, 26], axis=0) # get only 2018 stats
# population = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
# population = population.set_index('Metropolitan area')
# nhl_df['team'] = nhl_df['team'].apply(lambda x: x[:-1].strip() if x.endswith("*") else x.strip())
# nhl_df['area'] = nhl_df['team'].apply(lambda x: x.split(" ")[-1])
# nhl_df['area'] = nhl_df['area'].apply(lambda x: get_area(x))
# out = []
# for group, frame in nhl_df.groupby('area'):
#     total_wins = np.sum(pd.to_numeric(frame['W']))
#     total_losses = np.sum(pd.to_numeric(frame['L']))
#     total_matches = total_wins + total_losses
#     ratio = total_wins / total_matches
#     out_dict = {
#         'Area': group,
#         'Ratio': ratio
#     }
#     out.append(out_dict)
# new_df = pd.DataFrame(out)
# new_df = new_df.set_index('Area')
# out_df = pd.merge(new_df, population, how="inner", left_index=True, right_index=True)
# out_df['Population (2016 est.)[8]'] = pd.to_numeric(out_df['Population (2016 est.)[8]'])

# def nhl_correlation(): 
#     population_by_region = []  # pass in metropolitan area population from cities
#     win_loss_by_region = []  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
#     population_by_region = out_df['Population (2016 est.)[8]'].to_list()
#     win_loss_by_region = out_df['Ratio'].to_list()

#     assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
#     assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
#     return np.float64(stats.pearsonr(population_by_region, win_loss_by_region)[0])


# def get_nhl_data():
#     return out_df


# import pandas as pd
# import numpy as np
# import scipy.stats as stats
# import re

# def clear_data(string1):
#     if re.search(r'\[[a-z]* [0-9]+\]', string1) is None:
#         return string1
#     else:
#         return string1.replace(re.search(r'\[[a-z]* [0-9]+\]', string1).group(), '')


# def get_area(team):
#     for each in list(nhl_cities.index.values):
#         if team in each:
#             return nhl_cities.at[each, 'Metropolitan area']


# nhl_df=pd.read_csv("assets/nhl.csv")
# cities=pd.read_html("assets/wikipedia_data.html")[1]
# cities=cities.iloc[:-1,[0,3,5,6,7,8]]
# cities['NHL'] = cities['NHL'].apply(lambda x: clear_data(x))
# nhl_cities = cities[['Metropolitan area', 'NHL']].set_index('NHL')
# nhl_cities = nhl_cities.drop(['—', ''], axis=0)
# nhl_df = nhl_df[nhl_df['year'] == 2018].drop([0, 9, 18, 26], axis=0) # get only 2018 stats
# population = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
# population = population.set_index('Metropolitan area')
# nhl_df['team'] = nhl_df['team'].apply(lambda x: x[:-1].strip() if x.endswith("*") else x.strip())
# nhl_df['area'] = nhl_df['team'].apply(lambda x: x.split(" ")[-1])
# nhl_df['area'] = nhl_df['area'].apply(lambda x: get_area(x))
# out = []
# for group, frame in nhl_df.groupby('area'):
#     total_wins = np.sum(pd.to_numeric(frame['W']))
#     total_losses = np.sum(pd.to_numeric(frame['L']))
#     total_matches = total_wins + total_losses
#     ratio = total_wins / total_matches
#     out_dict = {
#         'Area': group,
#         'Ratio': ratio
#     }
#     out.append(out_dict)
# new_df = pd.DataFrame(out)
# new_df = new_df.set_index('Area')
# out_df = pd.merge(new_df, population, how="inner", left_index=True, right_index=True)
# out_df['Population (2016 est.)[8]'] = pd.to_numeric(out_df['Population (2016 est.)[8]'])

# def nhl_correlation(): 
#     population_by_region = []  # pass in metropolitan area population from cities
#     win_loss_by_region = []  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
#     population_by_region = out_df['Population (2016 est.)[8]'].to_list()
#     win_loss_by_region = out_df['Ratio'].to_list()

#     assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
#     assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
#     return np.float64(stats.pearsonr(population_by_region, win_loss_by_region)[0])


# def get_nhl_data():
#     return out_df

# import pandas as pd
# import numpy as np
# import scipy.stats as stats
# import re

# def clear_data(string1):
#     if re.search(r'\[[a-z]* [0-9]+\]', string1) is None:
#         return string1
#     else:
#         return string1.replace(re.search(r'\[[a-z]* [0-9]+\]', string1).group(), '')


# def get_area(team):
#     for each in list(nhl_cities.index.values):
#         if team in each:
#             return nhl_cities.at[each, 'Metropolitan area']


# nhl_df=pd.read_csv("assets/nhl.csv")
# cities=pd.read_html("assets/wikipedia_data.html")[1]
# cities=cities.iloc[:-1,[0,3,5,6,7,8]]
# cities['NHL'] = cities['NHL'].apply(lambda x: clear_data(x))
# nhl_cities = cities[['Metropolitan area', 'NHL']].set_index('NHL')
# nhl_cities = nhl_cities.drop(['—', ''], axis=0)
# nhl_df = nhl_df[nhl_df['year'] == 2018].drop([0, 9, 18, 26], axis=0) # get only 2018 stats
# population = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
# population = population.set_index('Metropolitan area')
# nhl_df['team'] = nhl_df['team'].apply(lambda x: x[:-1].strip() if x.endswith("*") else x.strip())
# nhl_df['area'] = nhl_df['team'].apply(lambda x: x.split(" ")[-1])
# nhl_df['area'] = nhl_df['area'].apply(lambda x: get_area(x))
# out = []
# for group, frame in nhl_df.groupby('area'):
#     total_wins = np.sum(pd.to_numeric(frame['W']))
#     total_losses = np.sum(pd.to_numeric(frame['L']))
#     total_matches = total_wins + total_losses
#     ratio = total_wins / total_matches
#     out_dict = {
#         'Area': group,
#         'Ratio': ratio
#     }
#     out.append(out_dict)
# new_df = pd.DataFrame(out)
# new_df = new_df.set_index('Area')
# out_df = pd.merge(new_df, population, how="inner", left_index=True, right_index=True)
# out_df['Population (2016 est.)[8]'] = pd.to_numeric(out_df['Population (2016 est.)[8]'])

# def nhl_correlation(): 
#     population_by_region = []  # pass in metropolitan area population from cities
#     win_loss_by_region = []  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
#     population_by_region = out_df['Population (2016 est.)[8]'].to_list()
#     win_loss_by_region = out_df['Ratio'].to_list()

#     assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
#     assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
#     return np.float64(stats.pearsonr(population_by_region, win_loss_by_region)[0])


# def get_nhl_data():
#     return out_df
# import pandas as pd
# import numpy as np
# import scipy.stats as stats
# import re

# nhl_df=pd.read_csv("assets/nhl.csv")
# cities=pd.read_html("assets/wikipedia_data.html")[1]
# cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# def nhl_correlation(): 
#     # YOUR CODE HERE
#     raise NotImplementedError()
    
#     population_by_region = [] # pass in metropolitan area population from cities
#     win_loss_by_region = [] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

#     assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
#     assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
#     return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
cities.rename(columns={"Population (2016 est.)[8]": "Population"},
              inplace=True)
cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "")
cities['MLB'] = cities['MLB'].str.replace(r"\[.*\]", "")
cities['NBA'] = cities['NBA'].str.replace(r"\[.*\]", "")
cities['NHL'] = cities['NHL'].str.replace(r"\[.*\]", "")

Big4='NBA'

def nba_correlation():
    
    team = cities[Big4].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area']=cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("",np.nan).replace("—",np.nan).dropna().reset_index().rename(columns = {"value":"team"})
    team=pd.merge(team,cities,how='left',on = 'Metropolitan area').iloc[:,1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team']=team['team'].str.replace('[\w.]*\ ','')

    _df=pd.read_csv("assets/"+str.lower(Big4)+".csv")
    _df = _df[_df['year'] == 2018]
    _df['team'] = _df['team'].str.replace(r'[\*]',"")
    _df['team'] = _df['team'].str.replace(r'\(\d*\)',"")
    _df['team'] = _df['team'].str.replace(r'[\xa0]',"")
    _df = _df[['team','W/L%']]
    _df['team'] = _df['team'].str.replace('[\w.]* ','')
    _df = _df.astype({'team': str,'W/L%': float})
    
    merge=pd.merge(team,_df,'outer', on = 'team')
    merge=merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})

    population_by_region = merge['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = merge['W/L%'] # pass in win/loss ratio from _df in the same order as cities["Metropolitan area"]   

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

#     return stats.pearsonr(population_by_region, win_loss_by_region)[0]
# nba_correlation()
    correlation_coefficient = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    
    print("NBA DataFrame:")
    print(merge)  # Print the merged DataFrame
    print("Correlation Coefficient:", correlation_coefficient)
    
    return correlation_coefficient

# nba_correlation()

# result = nba_correlation()
# rounded_result = round(result, 17)  # Round to match the expected number of decimal places
# print(rounded_result)


# import pandas as pd
# import numpy as np
# import scipy.stats as stats
# import re

# cities = pd.read_html("assets/wikipedia_data.html")[1]
# cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
# cities.rename(columns={"Population (2016 est.)[8]": "Population"},
#               inplace=True)
# cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "")
# cities['MLB'] = cities['MLB'].str.replace(r"\[.*\]", "")
# cities['NBA'] = cities['NBA'].str.replace(r"\[.*\]", "")
# cities['NHL'] = cities['NHL'].str.replace(r"\[.*\]", "")

# Big4='NBA'

# def nba_correlation():
    
#     team = cities[Big4].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
#     team['Metropolitan area']=cities['Metropolitan area']
#     team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("",np.nan).replace("—",np.nan).dropna().reset_index().rename(columns = {"value":"team"})
#     team=pd.merge(team,cities,how='left',on = 'Metropolitan area').iloc[:,1:4]
#     team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
#     team['team']=team['team'].str.replace('[\w.]*\ ','')

#     _df=pd.read_csv("assets/"+str.lower(Big4)+".csv")
#     _df = _df[_df['year'] == 2018]
#     _df['team'] = _df['team'].str.replace(r'[\*]',"")
#     _df['team'] = _df['team'].str.replace(r'\(\d*\)',"")
#     _df['team'] = _df['team'].str.replace(r'[\xa0]',"")
#     _df = _df[['team','W/L%']]
#     _df['team'] = _df['team'].str.replace('[\w.]* ','')
#     _df = _df.astype({'team': str,'W/L%': float})
    
#     merge=pd.merge(team,_df,'outer', on = 'team')
#     merge=merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})

#     population_by_region = merge['Population'] # pass in metropolitan area population from cities
#     win_loss_by_region = merge['W/L%'] # pass in win/loss ratio from _df in the same order as cities["Metropolitan area"]   

#     assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
#     assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

#     return stats.pearsonr(population_by_region, win_loss_by_region)[0]
# nba_correlation()

# import pandas as pd
# import numpy as np
# import scipy.stats as stats
# import re

# nba_df=pd.read_csv("assets/nba.csv")
# cities=pd.read_html("assets/wikipedia_data.html")[1]
# cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# def nba_correlation():
#     # YOUR CODE HERE
#     raise NotImplementedError()
    
#     population_by_region = [] # pass in metropolitan area population from cities
#     win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

#     assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
#     assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

#     return stats.pearsonr(population_by_region, win_loss_by_region)

In [3]:
nba_correlation()

NBA DataFrame:
                         W/L%  Population
Metropolitan area                        
Atlanta                 0.293   5789700.0
Boston                  0.671   4794447.0
Charlotte               0.439   2474314.0
Chicago                 0.329   9512999.0
Cleveland               0.610   2055612.0
Dallas–Fort Worth       0.293   7233323.0
Denver                  0.561   2853077.0
Detroit                 0.476   4297617.0
Houston                 0.793   6772470.0
Indianapolis            0.585   2004230.0
Los Angeles             0.512  13310447.0
Memphis                 0.268   1342842.0
Miami–Fort Lauderdale   0.537   6066387.0
Milwaukee               0.537   1572482.0
Minneapolis–Saint Paul  0.573   3551036.0
New Orleans             0.585   1268883.0
New York City           0.341  20153634.0
Oklahoma City           0.585   1373211.0
Orlando                 0.305   2441257.0
Philadelphia            0.634   6070500.0
Phoenix                 0.256   4661537.0
Portland           

-0.16087318003059875

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def get_area(team):
    for each in list(mlb_cities.index.values):
        if team in each:
            return mlb_cities.at[each, 'Metropolitan area']

def clear_data(string1):
    if re.search(r'\[[a-z]* [0-9]+\]', string1) is None:
        return string1
    else:
        return string1.replace(re.search(r'\[[a-z]* [0-9]+\]', string1).group(), '')


mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['MLB'] = cities['MLB'].apply(lambda x: clear_data(x))
mlb_cities = cities[['Metropolitan area', 'MLB']].set_index('MLB')
mlb_cities = mlb_cities.drop(['—', ''], axis=0)
mlb_df = mlb_df[mlb_df['year'] == 2018]  # get only 2018 stats no need of dropping rows
population = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
population = population.set_index('Metropolitan area')
mlb_df['area'] = mlb_df['team'].apply(lambda x: x.split(" ")[-1])
mlb_df['area'] = mlb_df['area'].apply(lambda x: get_area(x))
mlb_df.at[0, 'area'] = 'Boston'
out = []
for group, frame in mlb_df.groupby('area'):
    total_wins = np.sum(pd.to_numeric(frame['W']))
    total_losses = np.sum(pd.to_numeric(frame['L']))
    total_matches = total_wins + total_losses
    ratio = total_wins / total_matches
    out_dict = {
        'Area': group,
        'Ratio': ratio
    }
    out.append(out_dict)
new_df = pd.DataFrame(out)
new_df = new_df.set_index('Area')
out_df = pd.merge(new_df, population, how="inner", left_index=True, right_index=True)
out_df['Population (2016 est.)[8]'] = pd.to_numeric(out_df['Population (2016 est.)[8]'])


def mlb_correlation(): 
    population_by_region = []  # pass in metropolitan area population from cities
    win_loss_by_region = []  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    population_by_region = out_df['Population (2016 est.)[8]'].to_list()
    win_loss_by_region = out_df['Ratio'].to_list()

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return np.float64(stats.pearsonr(population_by_region, win_loss_by_region)[0])

def get_mlb_data():
    return out_df
# import pandas as pd
# import numpy as np
# import scipy.stats as stats
# import re


# def get_area(team):
#     for each in list(mlb_cities.index.values):
#         if team in each:
#             return mlb_cities.at[each, 'Metropolitan area']

# def clear_data(string1):
#     if re.search(r'\[[a-z]* [0-9]+\]', string1) is None:
#         return string1
#     else:
#         return string1.replace(re.search(r'\[[a-z]* [0-9]+\]', string1).group(), '')


# mlb_df=pd.read_csv("assets/mlb.csv")
# cities=pd.read_html("assets/wikipedia_data.html")[1]
# cities=cities.iloc[:-1,[0,3,5,6,7,8]]
# cities['MLB'] = cities['MLB'].apply(lambda x: clear_data(x))
# mlb_cities = cities[['Metropolitan area', 'MLB']].set_index('MLB')
# mlb_cities = mlb_cities.drop(['—', ''], axis=0)
# mlb_df = mlb_df[mlb_df['year'] == 2018]  # get only 2018 stats no need of dropping rows
# population = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
# population = population.set_index('Metropolitan area')
# mlb_df['area'] = mlb_df['team'].apply(lambda x: x.split(" ")[-1])
# mlb_df['area'] = mlb_df['area'].apply(lambda x: get_area(x))
# mlb_df.at[0, 'area'] = 'Boston'
# out = []
# for group, frame in mlb_df.groupby('area'):
#     total_wins = np.sum(pd.to_numeric(frame['W']))
#     total_losses = np.sum(pd.to_numeric(frame['L']))
#     total_matches = total_wins + total_losses
#     ratio = total_wins / total_matches
#     out_dict = {
#         'Area': group,
#         'Ratio': ratio
#     }
#     out.append(out_dict)
# new_df = pd.DataFrame(out)
# new_df = new_df.set_index('Area')
# out_df = pd.merge(new_df, population, how="inner", left_index=True, right_index=True)
# out_df['Population (2016 est.)[8]'] = pd.to_numeric(out_df['Population (2016 est.)[8]'])


# def mlb_correlation(): 
#     population_by_region = []  # pass in metropolitan area population from cities
#     win_loss_by_region = []  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
#     population_by_region = out_df['Population (2016 est.)[8]'].to_list()
#     win_loss_by_region = out_df['Ratio'].to_list()

#     assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
#     assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

#     return np.float64(stats.pearsonr(population_by_region, win_loss_by_region)[0])

# def get_mlb_data():
#     return out_df

# import pandas as pd
# import numpy as np
# import scipy.stats as stats
# import re

# mlb_df=pd.read_csv("assets/mlb.csv")
# cities=pd.read_html("assets/wikipedia_data.html")[1]
# cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# def mlb_correlation(): 
#     # YOUR CODE HERE
#     raise NotImplementedError()
    
#     population_by_region = [] # pass in metropolitan area population from cities
#     win_loss_by_region = [] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

#     assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
#     assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

#     return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def clear_data(string1):
    if re.search(r'\[[a-z]* [0-9]+\]', string1) is None:
        return string1
    else:
        return string1.replace(re.search(r'\[[a-z]* [0-9]+\]', string1).group(), '')


def clear_nba_data(string1):
    if re.search(r'\*|\+', string1) is None:
        return string1
    else:
        return string1.replace(re.search(r'\*|\+', string1).group(), '')


def get_area(team):
    for each in list(nfl_cities.index.values):
        if team in each:
            return nfl_cities.at[each, 'Metropolitan area']
        


cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['NFL'] = cities['NFL'].apply(lambda x: clear_data(x))
nfl_cities = cities[['Metropolitan area', 'NFL']].set_index('NFL')
nfl_cities = nfl_cities.drop(['—', ''], axis=0)
nfl_df = pd.read_csv("assets/nfl.csv")
nfl_df = nfl_df[nfl_df['year'] == 2018].drop([0, 5, 10, 15, 20, 25, 30, 35])  # get only 2018 stats
population = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
population = population.set_index('Metropolitan area')
nfl_df['team'] = nfl_df['team'].apply(lambda x: clear_nba_data(x))
nfl_df['area'] = nfl_df['team'].apply(lambda x: x.split(" ")[-1])
nfl_df['area'] = nfl_df['area'].apply(lambda x: get_area(x))
out = []
for group, frame in nfl_df.groupby('area'):
    total_wins = np.sum(pd.to_numeric(frame['W']))
    total_losses = np.sum(pd.to_numeric(frame['L']))
    total_matches = total_wins + total_losses
    ratio = total_wins / total_matches
    out_dict = {
        'Area': group,
        'Ratio': ratio
    }
    out.append(out_dict)
new_df = pd.DataFrame(out)
new_df = new_df.set_index('Area')
out_df = pd.merge(new_df, population, how="inner", left_index=True, right_index=True)
out_df['Population (2016 est.)[8]'] = pd.to_numeric(out_df['Population (2016 est.)[8]'])

def nfl_correlation(): 
    population_by_region = []  # pass in metropolitan area population from cities
    win_loss_by_region = []  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    population_by_region = out_df['Population (2016 est.)[8]'].to_list()
    win_loss_by_region = out_df['Ratio'].to_list()

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return np.float64(stats.pearsonr(population_by_region, win_loss_by_region)[0])

def get_nfl_data():
    return out_df


# -----------------------------------------------------------------------------------------------------------

# -----------------------------------------------------------------------------------------------------------------------

# import pandas as pd
# import numpy as np
# import scipy.stats as stats
# import re


# def clear_data(string1):
#     if re.search(r'\[[a-z]* [0-9]+\]', string1) is None:
#         return string1
#     else:
#         return string1.replace(re.search(r'\[[a-z]* [0-9]+\]', string1).group(), '')


# def clear_nba_data(string1):
#     if re.search(r'\*|\+', string1) is None:
#         return string1
#     else:
#         return string1.replace(re.search(r'\*|\+', string1).group(), '')


# def get_area(team):
#     for each in list(nfl_cities.index.values):
#         if team in each:
#             return nfl_cities.at[each, 'Metropolitan area']
        


# cities=pd.read_html("assets/wikipedia_data.html")[1]
# cities=cities.iloc[:-1,[0,3,5,6,7,8]]
# cities['NFL'] = cities['NFL'].apply(lambda x: clear_data(x))
# nfl_cities = cities[['Metropolitan area', 'NFL']].set_index('NFL')
# nfl_cities = nfl_cities.drop(['—', ''], axis=0)
# nfl_df = pd.read_csv("assets/nfl.csv")
# nfl_df = nfl_df[nfl_df['year'] == 2018].drop([0, 5, 10, 15, 20, 25, 30, 35])  # get only 2018 stats
# population = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
# population = population.set_index('Metropolitan area')
# nfl_df['team'] = nfl_df['team'].apply(lambda x: clear_nba_data(x))
# nfl_df['area'] = nfl_df['team'].apply(lambda x: x.split(" ")[-1])
# nfl_df['area'] = nfl_df['area'].apply(lambda x: get_area(x))
# out = []
# for group, frame in nfl_df.groupby('area'):
#     total_wins = np.sum(pd.to_numeric(frame['W']))
#     total_losses = np.sum(pd.to_numeric(frame['L']))
#     total_matches = total_wins + total_losses
#     ratio = total_wins / total_matches
#     out_dict = {
#         'Area': group,
#         'Ratio': ratio
#     }
#     out.append(out_dict)
# new_df = pd.DataFrame(out)
# new_df = new_df.set_index('Area')
# out_df = pd.merge(new_df, population, how="inner", left_index=True, right_index=True)
# out_df['Population (2016 est.)[8]'] = pd.to_numeric(out_df['Population (2016 est.)[8]'])

# def nfl_correlation(): 
#     population_by_region = []  # pass in metropolitan area population from cities
#     win_loss_by_region = []  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
#     population_by_region = out_df['Population (2016 est.)[8]'].to_list()
#     win_loss_by_region = out_df['Ratio'].to_list()

#     assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
#     assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

#     return np.float64(stats.pearsonr(population_by_region, win_loss_by_region)[0])

# def get_nfl_data():
#     return out_df

# import pandas as pd
# import numpy as np
# import scipy.stats as stats
# import re

# nfl_df=pd.read_csv("assets/nfl.csv")
# cities=pd.read_html("assets/wikipedia_data.html")[1]
# cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# def nfl_correlation(): 
#     # YOUR CODE HERE
#     raise NotImplementedError()
    
#     population_by_region = [] # pass in metropolitan area population from cities
#     win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

#     assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
#     assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

#     return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [15]:
# import pandas as pd
# import numpy as np
# import scipy.stats as stats
# import re

# # Load and preprocess the data
# cities = pd.read_html("assets/wikipedia_data.html")[1]
# cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
# cities.rename(columns={"Population (2016 est.)[8]": "Population"},
#               inplace=True)
# cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "")
# cities['MLB'] = cities['MLB'].str.replace(r"\[.*\]", "")
# cities['NBA'] = cities['NBA'].str.replace(r"\[.*\]", "")
# cities['NHL'] = cities['NHL'].str.replace(r"\[.*\]", "")

# # cities = pd.read_html("assets/wikipedia_data.html")[1]
# # cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
# # cities.rename(columns={"Population (2016 est.)[8]": "Population"},
# #               inplace=True)
# # cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "")
# # cities['MLB'] = cities['MLB'].str.replace(r"\[.*\]", "")
# # cities['NBA'] = cities['NBA'].str.replace(r"\[.*\]", "")
# # cities['NHL'] = cities['NHL'].str.replace(r"\[.*\]", "")


# def nhl_df():
#     Big4='NHL'
#     team = cities[Big4].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
#     team['Metropolitan area']=cities['Metropolitan area']
#     team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("",np.nan).replace("—",np.nan).dropna().reset_index().rename(columns = {"value":"team"})
#     team=pd.merge(team,cities,how='left',on = 'Metropolitan area').iloc[:,1:4]
#     team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
#     team['team']=team['team'].str.replace('[\w.]*\ ','')
    
#     _df=pd.read_csv("assets/"+str.lower(Big4)+".csv")
#     _df = _df[_df['year'] == 2018]
#     _df['team'] = _df['team'].str.replace(r'\*',"")
#     _df = _df[['team','W','L']]

#     dropList=[]
#     for i in range(_df.shape[0]):
#         row=_df.iloc[i]
#         if row['team']==row['W'] and row['L']==row['W']:
#             dropList.append(i)
#     _df=_df.drop(dropList)

#     _df['team'] = _df['team'].str.replace('[\w.]* ','')
#     _df = _df.astype({'team': str,'W': int, 'L': int})
#     _df['W/L%'] = _df['W']/(_df['W']+_df['L'])
    
#     merge=pd.merge(team,_df,'inner', on = 'team')
#     merge=merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})  

#     return merge[['W/L%']]



# def nba_df():
#     Big4='NBA'
#     team = cities[Big4].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
#     team['Metropolitan area']=cities['Metropolitan area']
#     team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("",np.nan).replace("—",np.nan).dropna().reset_index().rename(columns = {"value":"team"})
#     team=pd.merge(team,cities,how='left',on = 'Metropolitan area').iloc[:,1:4]
#     team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
#     team['team']=team['team'].str.replace('[\w.]*\ ','')

#     _df=pd.read_csv("assets/"+str.lower(Big4)+".csv")
#     _df = _df[_df['year'] == 2018]
#     _df['team'] = _df['team'].str.replace(r'[\*]',"")
#     _df['team'] = _df['team'].str.replace(r'\(\d*\)',"")
#     _df['team'] = _df['team'].str.replace(r'[\xa0]',"")
#     _df = _df[['team','W/L%']]
#     _df['team'] = _df['team'].str.replace('[\w.]* ','')
#     _df = _df.astype({'team': str,'W/L%': float})
    
#     merge=pd.merge(team,_df,'outer', on = 'team')
#     merge=merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})
#     return merge[['W/L%']]



# # def mlb_df(): 
# #     Big4='MLB'
# #     team = cities[Big4].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
# #     team['Metropolitan area']=cities['Metropolitan area']
# #     team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("",np.nan).replace("—",np.nan).dropna().reset_index().rename(columns = {"value":"team"})
# #     team=pd.merge(team,cities,how='left',on = 'Metropolitan area').iloc[:,1:4]
# #     team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
# #     team['team']=team['team'].str.replace('\ Sox','Sox')
# #     team['team']=team['team'].str.replace('[\w.]*\ ','')

# #     _df=pd.read_csv("assets/"+str.lower(Big4)+".csv")
# #     _df = _df[_df['year'] == 2018]
# #     _df['team'] = _df['team'].str.replace(r'[\*]',"")
# #     _df['team'] = _df['team'].str.replace(r'\(\d*\)',"")
# #     _df['team'] = _df['team'].str.replace(r'[\xa0]',"")
# #     _df = _df[['team','W-L%']]
# #     _df.rename(columns={"W-L%": "W/L%"},inplace=True)
# #     _df['team']=_df['team'].str.replace('\ Sox','Sox')
# #     _df['team'] = _df['team'].str.replace('[\w.]* ','')
# #     _df = _df.astype({'team': str,'W/L%': float})
    
# #     merge=pd.merge(team,_df,'outer', on = 'team')
# #     merge=merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})

# #     return merge[['W/L%']]

# def mlb_df():
#     Big4 = 'MLB'
#     team = cities[Big4].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0_9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
#     team['Metropolitan area'] = cities['Metropolitan area']
#     team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("", np.nan).replace("—", np.nan).dropna().reset_index().rename(columns={"value": "team"})
#     team = pd.merge(team, cities, how='left', on='Metropolitan area').iloc[:, 1:4]
#     team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
#     team['team'] = team['team'].str.replace('\ Sox', 'Sox')
#     team['team'] = team['team'].str.replace('[\w.]*\ ', '')

#     _df = pd.read_csv("assets/" + str.lower(Big4) + ".csv")
#     _df = _df[_df['year'] == 2018]
#     _df['team'] = _df['team'].str.replace(r'[\*]', "")
#     _df['team'] = _df['team'].str.replace(r'\(\d*\)', "")
#     _df['team'] = _df['team'].str.replace(r'[\xa0]', "")
#     _df = _df[['team', 'W-L%']]
#     _df.rename(columns={"W-L%": "W/L%"}, inplace=True)
#     _df['team'] = _df['team'].str.replace('\ Sox', 'Sox')
#     _df['team'] = _df['team'].str.replace('[\w.]* ', '')

#     merge = pd.merge(team, _df, on='team', how='left')
#     merge = merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})

#     return merge[['W/L%']]


# def nfl_df(): 
#     Big4='NFL'
#     team = cities[Big4].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
#     team['Metropolitan area']=cities['Metropolitan area']
#     team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("",np.nan).replace("—",np.nan).dropna().reset_index().rename(columns = {"value":"team"})
#     team=pd.merge(team,cities,how='left',on = 'Metropolitan area').iloc[:,1:4]
#     team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
#     team['team']=team['team'].str.replace('[\w.]*\ ','')
    
#     _df=pd.read_csv("assets/"+str.lower(Big4)+".csv")
#     _df = _df[_df['year'] == 2018]
#     _df['team'] = _df['team'].str.replace(r'[\*]',"")
#     _df['team'] = _df['team'].str.replace(r'\(\d*\)',"")
#     _df['team'] = _df['team'].str.replace(r'[\xa0]',"")
#     _df = _df[['team','W-L%']]
#     _df.rename(columns={"W-L%": "W/L%"},inplace=True)
#     dropList=[]
#     for i in range(_df.shape[0]):
#         row=_df.iloc[i]
#         if row['team']==row['W/L%']:
#             dropList.append(i)
#     _df=_df.drop(dropList)

#     _df['team'] = _df['team'].str.replace('[\w.]* ','')
#     _df['team'] = _df['team'].str.replace('+','')
#     _df = _df.astype({'team': str,'W/L%': float})
    
#     merge=pd.merge(team,_df,'outer', on = 'team')
#     merge=merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})

#     return merge[['W/L%']]

# def create_df(sport):
#     if sport =='NFL':
#         return nfl_df()
#     elif sport =='NBA':
#         return nba_df()
#     elif sport =='NHL':
#         return nhl_df()
#     elif sport =='MLB':
#         return mlb_df()
#     else:
#         print("ERROR with intput!")


# def sports_team_performance():
#     # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
#     # df.loc["NFL","NFL"] should return np.nan
#     sports = ['NFL', 'NBA', 'NHL', 'MLB']
#     p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
#     for i in sports:
#         for j in sports:
#             if i !=j :
#                 merge=pd.merge(create_df(i),create_df(j),'inner', on = ['Metropolitan area'])
#                 p_values.loc[i, j]=stats.ttest_rel(merge['W/L%_x'],merge['W/L%_y'])[1]
                
#                 print(f"P-value for {i}-{j}: {p_values.loc[i, j]}")

#     assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
#     assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
#     return p_values

# import pandas as pd
# from scipy.stats import ttest_rel

# # Load the required dataframes and prepare the data
# # Make sure you have already loaded the necessary dataframes: wikipedia_data, nfl_df, mlb_df, nba_df, nhl_df

# # Calculate the win/loss ratio for each sport
# def calculate_win_loss_ratio(df):
#     return df["W"] / (df["W"] + df["L"])

# # Calculate average win/loss ratio for cities with multiple teams of a single sport
# def calculate_average_win_loss_ratio(grouped_teams):
#     return grouped_teams.mean()

# # Group the teams by their metropolitan area and sport
# grouped_teams = wikipedia_data.groupby(["Metropolitan area", "Sport"]).apply(calculate_win_loss_ratio)
# average_win_loss_ratio = grouped_teams.groupby(["Metropolitan area", "Sport"]).apply(calculate_average_win_loss_ratio)

# # Filter out cities with only one team in a sport
# average_win_loss_ratio = average_win_loss_ratio[average_win_loss_ratio.groupby(["Metropolitan area", "Sport"]).transform(len) > 1]

# # Get a list of unique sports
# sports = average_win_loss_ratio.index.get_level_values("Sport").unique()

# # Initialize a DataFrame to store the p-values
# p_values = pd.DataFrame(index=sports, columns=sports)

# # Perform paired t-tests and fill in the p-values
# for sport1 in sports:
#     for sport2 in sports:
#         if sport1 != sport2:
#             teams1 = average_win_loss_ratio[average_win_loss_ratio.index.get_level_values("Sport") == sport1]["W/L"]
#             teams2 = average_win_loss_ratio[average_win_loss_ratio.index.get_level_values("Sport") == sport2]["W/L"]
#             _, p_value = ttest_rel(teams1, teams2)
#             p_values.loc[sport1, sport2] = p_value

# # Now you can check which p-values are significant and reject the null hypothesis
# print("P-values:")
# print(p_values)

# # Assertions
# assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
# assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"


In [16]:
# print(sports_team_performance())
# sports_team_performance()

In [17]:
# (nfl_df, nba_df, nhl_df, mlb_df) = clean_dfs()   
# leagues = {"NFL": nfl_df, "NBA": nba_df, "NHL": nhl_df, "MLB": mlb_df}
# p_values_dict = calculate_p_values(leagues)
# p_values_dict

In [18]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
cities.rename(columns={"Population (2016 est.)[8]": "Population"},
              inplace=True)
cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "")
cities['MLB'] = cities['MLB'].str.replace(r"\[.*\]", "")
cities['NBA'] = cities['NBA'].str.replace(r"\[.*\]", "")
cities['NHL'] = cities['NHL'].str.replace(r"\[.*\]", "")



def nhl_df():
    Big4='NHL'
    team = cities[Big4].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area']=cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("",np.nan).replace("—",np.nan).dropna().reset_index().rename(columns = {"value":"team"})
    team=pd.merge(team,cities,how='left',on = 'Metropolitan area').iloc[:,1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team']=team['team'].str.replace('[\w.]*\ ','')
    
    _df=pd.read_csv("assets/"+str.lower(Big4)+".csv")
    _df = _df[_df['year'] == 2018]
    _df['team'] = _df['team'].str.replace(r'\*',"")
    _df = _df[['team','W','L']]

    dropList=[]
    for i in range(_df.shape[0]):
        row=_df.iloc[i]
        if row['team']==row['W'] and row['L']==row['W']:
            dropList.append(i)
    _df=_df.drop(dropList)

    _df['team'] = _df['team'].str.replace('[\w.]* ','')
    _df = _df.astype({'team': str,'W': int, 'L': int})
    _df['W/L%'] = _df['W']/(_df['W']+_df['L'])
    
    merge=pd.merge(team,_df,'inner', on = 'team')
    merge=merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})  

    return merge[['W/L%']]



def nba_df():
    Big4='NBA'
    team = cities[Big4].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area']=cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("",np.nan).replace("—",np.nan).dropna().reset_index().rename(columns = {"value":"team"})
    team=pd.merge(team,cities,how='left',on = 'Metropolitan area').iloc[:,1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team']=team['team'].str.replace('[\w.]*\ ','')

    _df=pd.read_csv("assets/"+str.lower(Big4)+".csv")
    _df = _df[_df['year'] == 2018]
    _df['team'] = _df['team'].str.replace(r'[\*]',"")
    _df['team'] = _df['team'].str.replace(r'\(\d*\)',"")
    _df['team'] = _df['team'].str.replace(r'[\xa0]',"")
    _df = _df[['team','W/L%']]
    _df['team'] = _df['team'].str.replace('[\w.]* ','')
    _df = _df.astype({'team': str,'W/L%': float})
    
    merge=pd.merge(team,_df,'outer', on = 'team')
    merge=merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})
    return merge[['W/L%']]



def mlb_df(): 
    Big4='MLB'
    team = cities[Big4].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area']=cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("",np.nan).replace("—",np.nan).dropna().reset_index().rename(columns = {"value":"team"})
    team=pd.merge(team,cities,how='left',on = 'Metropolitan area').iloc[:,1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team']=team['team'].str.replace('\ Sox','Sox')
    team['team']=team['team'].str.replace('[\w.]*\ ','')

    _df=pd.read_csv("assets/"+str.lower(Big4)+".csv")
    _df = _df[_df['year'] == 2018]
    _df['team'] = _df['team'].str.replace(r'[\*]',"")
    _df['team'] = _df['team'].str.replace(r'\(\d*\)',"")
    _df['team'] = _df['team'].str.replace(r'[\xa0]',"")
    _df = _df[['team','W-L%']]
    _df.rename(columns={"W-L%": "W/L%"},inplace=True)
    _df['team']=_df['team'].str.replace('\ Sox','Sox')
    _df['team'] = _df['team'].str.replace('[\w.]* ','')
    _df = _df.astype({'team': str,'W/L%': float})
    
    merge=pd.merge(team,_df,'outer', on = 'team')
    merge=merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})

    return merge[['W/L%']]

def nfl_df(): 
    Big4='NFL'
    team = cities[Big4].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area']=cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("",np.nan).replace("—",np.nan).dropna().reset_index().rename(columns = {"value":"team"})
    team=pd.merge(team,cities,how='left',on = 'Metropolitan area').iloc[:,1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team']=team['team'].str.replace('[\w.]*\ ','')
    
    _df=pd.read_csv("assets/"+str.lower(Big4)+".csv")
    _df = _df[_df['year'] == 2018]
    _df['team'] = _df['team'].str.replace(r'[\*]',"")
    _df['team'] = _df['team'].str.replace(r'\(\d*\)',"")
    _df['team'] = _df['team'].str.replace(r'[\xa0]',"")
    _df = _df[['team','W-L%']]
    _df.rename(columns={"W-L%": "W/L%"},inplace=True)
    dropList=[]
    for i in range(_df.shape[0]):
        row=_df.iloc[i]
        if row['team']==row['W/L%']:
            dropList.append(i)
    _df=_df.drop(dropList)

    _df['team'] = _df['team'].str.replace('[\w.]* ','')
    _df['team'] = _df['team'].str.replace('+','')
    _df = _df.astype({'team': str,'W/L%': float})
    
    merge=pd.merge(team,_df,'outer', on = 'team')
    merge=merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})

    return merge[['W/L%']]

def create_df(sport):
    if sport =='NFL':
        return nfl_df()
    elif sport =='NBA':
        return nba_df()
    elif sport =='NHL':
        return nhl_df()
    elif sport =='MLB':
        return mlb_df()
    else:
        print("ERROR with intput!")

        
def sports_team_performance():
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    for i in sports:
        for j in sports:
            if i !=j :
                merge=pd.merge(create_df(i),create_df(j),'inner', on = ['Metropolitan area'])
                p_values.loc[i, j]=stats.ttest_rel(merge['W/L%_x'],merge['W/L%_y'])[1]
                
                print(f"P-value for {i}-{j}: {p_values.loc[i, j]}")

    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    # assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values


In [19]:
sports_team_performance()

P-value for NFL-NBA: 0.9759717777734693
P-value for NFL-NHL: 0.022377486385610375
P-value for NFL-MLB: nan
P-value for NBA-NFL: 0.9759717777734693
P-value for NBA-NHL: 0.023340858718500748
P-value for NBA-MLB: nan
P-value for NHL-NFL: 0.022377486385610375
P-value for NHL-NBA: 0.023340858718500748
P-value for NHL-MLB: nan
P-value for MLB-NFL: nan
P-value for MLB-NBA: nan
P-value for MLB-NHL: nan


,NFL,NBA,NHL,MLB
NFL,NaN,0.975972,0.022377,NaN
NBA,0.975972,NaN,0.023341,NaN
NHL,0.022377,0.023341,NaN,NaN
MLB,NaN,NaN,NaN,NaN


In [20]:
(nfl_df, nba_df, nhl_df, mlb_df) = clean_dfs()   
leagues = {"NFL": nfl_df, "NBA": nba_df, "NHL": nhl_df, "MLB": mlb_df}
p_values_dict = calculate_p_values(leagues)
p_values_dict

,NFL,NBA,NHL,MLB
NFL,NaN,0.692438,0.053925,0.742217
NBA,0.692438,NaN,0.087494,0.639435
NHL,0.053925,0.087494,NaN,0.001001
MLB,0.742217,0.639435,0.001001,NaN


In [21]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def clean_nhl_df():
    # load data
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    # cleaning the cities dataframe
    cities["NHL"] = cities["NHL"].apply(lambda x: re.sub(r"\[.+\]", "", x))
    cities["NHL"] = cities["NHL"].replace({"RangersIslandersDevils": "Rangers,Islanders,Devils",
                                           "KingsDucks": "Kings,Ducks",
                                           "Red Wings": "Red,Wings", 
                                           "Maple Leafs": "Maple,Leafs", 
                                           "Blue Jackets": "Blue,Jackets",
                                           "Golden Knights": "Golden,Knights" })
    cities["NHL"] = cities["NHL"].apply(lambda x: x.split(","))
    cities = cities.explode("NHL")

    # cleaning the nhl_df dataframe
    nhl_df = nhl_df[nhl_df["year"] == 2018]
    nhl_df["team"] = nhl_df["team"].apply(lambda x: x.replace("*", ""))
    nhl_df["team"] = nhl_df["team"].apply(lambda x: x.split(" ")[-1])

    # merge the dataframes
    df = pd.merge(cities, nhl_df, left_on="NHL", right_on="team")
    df = df[["Metropolitan area", "Population (2016 est.)[8]", "NHL", "team", "W", "L"]]
    df["W-L%"] = df["W"].astype("int")/(df["W"].astype("int") + df["L"].astype("int"))
    df["Population (2016 est.)[8]"] = df["Population (2016 est.)[8]"].astype("float")
    df["W-L%"] = df["W-L%"].astype("float")

    # drop duplicated columns
    df.loc[df["Metropolitan area"] == "New York City", "W-L%"] = 0.5182013333333334 # mean of NY W-L%
    df.loc[df["Metropolitan area"] == "Los Angeles", "W-L%"] = 0.6228945 # mean of LA W-L%
    df = df.drop_duplicates(subset="Metropolitan area").reset_index()
    df = df.drop(columns="index")
    
    return df

def nhl_correlation(): 
    # YOUR CODE HERE
#     raise NotImplementedError()
    df = clean_nhl_df()

    population_by_region = df["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = df["W-L%"] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    result = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return result[0]


In [22]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def clean_nba_df():
    # load data
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # cleaning the cities dataframe
    cities["NBA"] = cities["NBA"].apply(lambda x: re.sub(r"\[.+\]", "", x))
    cities["NBA"] = cities["NBA"].replace({"KnicksNets": "Knicks,Nets",
                                           "LakersClippers": "Lakers,Clippers",
                                           "Trail Blazers": "Trail,Blazers"})
    cities["NBA"] = cities["NBA"].apply(lambda x: x.split(","))
    cities = cities.explode("NBA")

    # cleaning the nhl_df dataframe
    nba_df = nba_df[nba_df["year"] == 2018]
    nba_df["team"] = nba_df["team"].apply(lambda x: re.sub(r"(\*)*\s\(\d+\)", "", x))
    nba_df["team"] = nba_df["team"].apply(lambda x: x.split(" ")[-1])

    # merge the dataframes
    df = pd.merge(cities, nba_df, left_on="NBA", right_on="team")
    df = df.rename(columns={"W/L%": "W-L%"})
    df = df[["Metropolitan area", "Population (2016 est.)[8]", "NBA", "team", "W", "L", "W-L%"]]
    df["Population (2016 est.)[8]"] = df["Population (2016 est.)[8]"].astype("float")
    df["W-L%"] = df["W-L%"].astype("float")
    
    # drop duplicated columns
    df.loc[df["Metropolitan area"] == "New York City", "W-L%"] = 0.34750000000000003 # mean of NY W-L%
    df.loc[df["Metropolitan area"] == "Los Angeles", "W-L%"] = 0.46950000000000003 # mean of LA W-L%
    df = df.drop_duplicates(subset="Metropolitan area").reset_index()
    df = df.drop(columns="index")
    
    return df

def nba_correlation():
    # YOUR CODE HERE
#     raise NotImplementedError()
    df = clean_nba_df()

    population_by_region = df["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = df["W-L%"] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    result = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return result[0]


In [23]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def clean_mlb_df():
    # load data
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    # cleaning the cities dataframe
    cities["MLB"] = cities["MLB"].apply(lambda x: re.sub(r"\[.+\]", "", x))
    cities["MLB"] = cities["MLB"].replace({"Blue Jays": "Blue,Jays", 
                                           "CubsWhite Sox": "Cubs,White,Sox", 
                                           "DodgersAngels": "Dodgers,Angels", 
                                           "GiantsAthletics": "Giants,Athletics", 
                                           "YankeesMets": "Yankees,Mets",
                                           "Red Sox": "Red,Sox"})
    cities["MLB"] = cities["MLB"].apply(lambda x: x.split(","))
    cities = cities.explode("MLB")

    # cleaning the nhl_df dataframe
    mlb_df = mlb_df[mlb_df["year"] == 2018]
    mlb_df["team"] = mlb_df["team"].apply(lambda x: x.split(" ")[-1])

    # merge the dataframes
    df = pd.merge(cities, mlb_df, left_on="MLB", right_on="team")
    df = df[["Metropolitan area", "Population (2016 est.)[8]", "MLB", "team", "W", "L", "W-L%"]]
    df["Population (2016 est.)[8]"] = df["Population (2016 est.)[8]"].astype("float")
    df["W-L%"] = df["W-L%"].astype("float")

    # drop duplicated columns
    df.loc[df["Metropolitan area"] == "New York City", "W-L%"] = 0.546 # mean of NY W-L%
    df.loc[df["Metropolitan area"] == "Los Angeles", "W-L%"] = 0.5289999999999999 # mean of LA W-L%
    df.loc[df["Metropolitan area"] == "San Francisco Bay Area", "W-L%"] = 0.525 # mean of SF W-L%
    df.loc[df["Metropolitan area"] == "Chicago", "W-L%"] = 0.482769 # mean of CH W-L%
    df.loc[df["Metropolitan area"] == "Boston", "W-L%"] = 0.666667 # mean of BO W-L%
    df = df.drop_duplicates(subset="Metropolitan area").reset_index()
    df = df.drop(columns="index")
    
    return df

def mlb_correlation(): 
    # YOUR CODE HERE
#     raise NotImplementedError()
    df = clean_mlb_df()

    population_by_region = df["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = df["W-L%"] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    result = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return result[0]


In [24]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def clean_nfl_df():
    # load data
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    # cleaning the cities dataframe
    cities["NFL"] = cities["NFL"].apply(lambda x: re.sub(r"\[.+\]", "", x))
    cities["NFL"] = cities["NFL"].replace({"GiantsJets": "Giants,Jets",
                                           "RamsChargers": "Rams,Chargers",
                                           "49ersRaiders": "49ers,Raiders"
                                           })
    cities["NFL"] = cities["NFL"].apply(lambda x: x.split(","))
    cities = cities.explode("NFL")

    # cleaning the nhl_df dataframe
    nfl_df = nfl_df[nfl_df["year"] == 2018]
    nfl_df["team"] = nfl_df["team"].apply(lambda x: re.sub(r"(\*|\+)", "", x))
    nfl_df["team"] = nfl_df["team"].apply(lambda x: x.split(" ")[-1])

    # merge the dataframes
    df = pd.merge(cities, nfl_df, left_on="NFL", right_on="team")
    df = df[["Metropolitan area", "Population (2016 est.)[8]", "NFL", "team", "W", "L", "W-L%"]]
    df["Population (2016 est.)[8]"] = df["Population (2016 est.)[8]"].astype("float")
    df["W-L%"] = df["W-L%"].astype("float")

    # drop duplicated columns
    df.loc[df["Metropolitan area"] == "New York City", "W-L%"] = 0.2815 # mean of NY W-L%
    df.loc[df["Metropolitan area"] == "Los Angeles", "W-L%"] = 0.7815 # mean of LA W-L%
    df.loc[df["Metropolitan area"] == "San Francisco Bay Area", "W-L%"] = 0.25 # mean of SF W-L%
    df = df.drop_duplicates(subset="Metropolitan area").reset_index()
    df = df.drop(columns="index")
    
    return df

def nfl_correlation(): 
    # YOUR CODE HERE
#     raise NotImplementedError()
    df = clean_nfl_df()
    
    population_by_region = df["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = df["W-L%"] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    result = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return result[0]


In [25]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# clean the dataframes
def clean_dfs():
    nfl_df = clean_nfl_df()
    nba_df = clean_nba_df()
    nhl_df = clean_nhl_df()   
    mlb_df = clean_mlb_df()
    
    
    nfl_df = nfl_df[["Metropolitan area", "W-L%"]]
    nba_df = nba_df[["Metropolitan area", "W-L%"]]
    nhl_df = nhl_df[["Metropolitan area", "W-L%"]]  
    mlb_df = mlb_df[["Metropolitan area", "W-L%"]]
    
    return (nfl_df, nba_df, nhl_df, mlb_df)

def calculate_p_values(leagues):
    p_values = pd.DataFrame(columns=leagues.keys(), index=leagues.keys())
    for (league_name1, league_df1) in leagues.items():
        for (league_name2, league_df2) in leagues.items():
            merged_league = pd.merge(league_df1, league_df2, on="Metropolitan area")
            p_values.loc[league_name1, league_name2] = stats.ttest_rel(merged_league["W-L%_x"], merged_league["W-L%_y"])[1]
            
    return p_values

def sports_team_performance():
    # YOUR CODE HERE
#     raise NotImplementedError()
    (nfl_df, nba_df, nhl_df, mlb_df) = clean_dfs()
    
    leagues = {"NFL": nfl_df, "NBA": nba_df, "NHL": nhl_df, "MLB": mlb_df}
    p_values_dict = calculate_p_values(leagues)
    p_values = pd.DataFrame(p_values_dict).astype("float")
    
#     # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
#     # df.loc["NFL","NFL"] should return np.nan
#     sports = ['NFL', 'NBA', 'NHL', 'MLB']
#     p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NFL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

In [26]:
sports_team_performance()

AssertionError: The NBA-NFL p-value should be around 0.02

In [27]:
(nfl_df, nba_df, nhl_df, mlb_df) = clean_dfs()   
leagues = {"NFL": nfl_df, "NBA": nba_df, "NHL": nhl_df, "MLB": mlb_df}
p_values_dict = calculate_p_values(leagues)
p_values_dict

,NFL,NBA,NHL,MLB
NFL,NaN,0.692438,0.053925,0.742217
NBA,0.692438,NaN,0.087494,0.639435
NHL,0.053925,0.087494,NaN,0.001001
MLB,0.742217,0.639435,0.001001,NaN


In [28]:
print(p_values.loc["NBA", "NHL"])
print(p_values.loc["MLB", "NFL"])

NameError: name 'p_values' is not defined

In [29]:
(nhl_df, nba_df, mlb_df, nfl_df) = clean_dfs()
leagues = {"NFL": nfl_df, "NBA": nba_df, "NHL": nhl_df, "MLB": mlb_df}
for (league_name1, league_df1) in leagues.items():
        for (league_name2, league_df2) in leagues.items():
            merged_league = pd.merge(league_df1, league_df2, on="Metropolitan area")
            print(f"{league_name1}_{league_name2}_{len(merged_league)}")


NFL_NFL_22
NFL_NBA_14
NFL_NHL_18
NFL_MLB_13
NBA_NFL_14
NBA_NBA_26
NBA_NHL_16
NBA_MLB_12
NHL_NFL_18
NHL_NBA_16
NHL_NHL_26
NHL_MLB_14
MLB_NFL_13
MLB_NBA_12
MLB_NHL_14
MLB_MLB_26
